In [1]:
from cox_nnet import *
import numpy
import sklearn
from sklearn import model_selection as cross_validation

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
F:\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
# load data
x = numpy.loadtxt(fname="E:/中山三院/CKD/usa/MDRD_NIDDK_DEVELOPEMENT_output.csv",delimiter=",",skiprows=0)
ytime = numpy.loadtxt(fname="E:/中山三院/CKD/usa/MDRD_NIDDK_DEVELOPEMENT -time.csv",delimiter=",",skiprows=0)
ystatus = numpy.loadtxt(fname="E:/中山三院/CKD/usa/MDRD_NIDDK_DEVELOPEMENT - event.csv",delimiter=",",skiprows=0)

# split into test/train sets
x_train, x_test, ytime_train, ytime_test, ystatus_train, ystatus_test = \
    cross_validation.train_test_split(x, ytime, ystatus, 
    test_size = 0.2, random_state = 100)

#Define parameters
model_params = dict(node_map = None, input_split = None)
search_params = dict(method = "nesterov", learning_rate=0.01, momentum=0.9,
    max_iter=2000, stop_threshold=0.995, patience=1000, patience_incr=2, rand_seed = 123,
    eval_step=23, lr_decay = 0.9, lr_growth = 1.0)
cv_params = dict(cv_seed=1, n_folds=5, cv_metric = "loglikelihood",
    L2_range = numpy.arange(-4.5,1,0.5))

#cross validate training set to determine lambda parameters
cv_likelihoods, L2_reg_params, mean_cvpl = L2CVProfile(x_train,ytime_train,ystatus_train,
    model_params,search_params,cv_params, verbose=False)

TypeError: __init__() got an unexpected keyword argument 'n_folds'

In [ ]:
#Build the model
L2_reg = L2_reg_params[numpy.argmax(mean_cvpl)]
model_params = dict(node_map = None, input_split = None, L2_reg=numpy.exp(L2_reg))
model, cost_iter = trainCoxMlp(x_train, ytime_train, ystatus_train, 
    model_params, search_params, verbose=True)
theta = model.predictNewData(x_test)